<a href="https://colab.research.google.com/github/perfectism13/learning_colab/blob/master/keras_video_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#用keras实现动作分类

##前期准备

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.chdir(r'/content/drive/My Drive/colab/keras-video-classification/keras-video-classification')
print(os.getcwd())
!ls

In [0]:
!apt-get install p7zip

In [0]:
!7z x sports-type-classifier-data.7z

In [0]:
!ls
!tree --dirsfirst --filelimit 50

##开始

In [0]:
# USAGE
# python train.py --dataset Sports-Type-Classifier/data --model model/activity.model --label-bin model/lb.pickle --epochs 50

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os

# construct the argument parser and parse the arguments
# 构建python文件运行时的命令行输入
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
	help="path to input dataset")
ap.add_argument("-m", "--model", required=True,
	help="path to output serialized model")
ap.add_argument("-l", "--label-bin", required=True,
	help="path to output label binarizer")
ap.add_argument("-e", "--epochs", type=int, default=25,
	help="# of epochs to train our network for")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
	help="path to output loss/accuracy plot")
args = vars(ap.parse_args())

# initialize the set of labels from the spots activity dataset we are
# going to train our network on
LABELS = set(["weight_lifting", "tennis", "football"])

# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")
# 通过imtuils的paths类获取dataset对应路径下所有图片的路径
imagePaths = list(paths.list_images(args["dataset"])) 
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# if the label of the current image is not part of of the labels
	# are interested in, then ignore the image
	# 跳过不参与训练的标签对应的图片
	if label not in LABELS:
		continue

	# load the image, convert it to RGB channel ordering, and resize
	# it to be a fixed 224x224 pixels, ignoring aspect ratio
	image = cv2.imread(imagePath)
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))

	# update the data and labels lists, respectively
	# 储存训练标签和图片
	data.append(image)
	labels.append(label)

# convert the data and labels to NumPy arrays
# 将数据和标签转换成numpy数组
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
# 通过二值元素数组来对标签进行一键编码
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
# 将整个数据的%75用来训练剩下的用来检验
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=42)

# initialize the training data augmentation object
# 声明训练时的数据增强操作
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
# 测验时无数据增强操作
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
# 设置训练和测试时用来归一化的图片每个通道的均值
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
# 通过keras.applications来获取预训练的resnet50模型
baseModel = ResNet50(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
# 定义basemodel前的全连接层
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
# 在basemodel前加上一个全连接层来微调
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process
# 训练时将basemodel里的全部参数冻结
for layer in baseModel.layers:
	layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
# 定义初始学习率和优化方法来编译模型
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
# 使用fit_generator类来对逐批生成的数据进行训练
print("[INFO] training head...")
H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=32),
	steps_per_epoch=len(trainX) // 32,
	validation_data=valAug.flow(testX, testY),
	validation_steps=len(testX) // 32,
	epochs=args["epochs"])

# evaluate the network
# 使用sklearn的classification_report类查看预测效果
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), 
	predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
# 打印训练loss和准确性等
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left") # 标签说明放在左下角
plt.savefig(args["plot"])

# serialize the model to disk
# 讲训练好的模型保存
print("[INFO] serializing network...")
model.save(args["model"])

# serialize the label binarizer to disk
# 保存标签二值化器
f = open(args["label_bin"], "wb")
f.write(pickle.dumps(lb))
f.close()

In [0]:
!python train.py --dataset data --model output/activity.model \
	--label-bin output/lb.pickle --epochs 50

Using TensorFlow backend.
[INFO] loading images...
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile






2019-12-14 08:24:02.011318: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-12-14 08:24:02.013846: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2306840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-14 08:24:02.013885: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-12-14 08:24:02.018815: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-12-14 08:24:02.152830: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read f

##用滚动预测平均的方式应用模型到视频分类中

In [0]:
# USAGE
# python predict_video.py --model model/activity.model --label-bin model/lb.pickle --input example_clips/lifting.mp4 --output output/lifting_128avg.avi --size 128

# import the necessary packages
from keras.models import load_model
from collections import deque #用来实现滑动平均
import numpy as np
import argparse
import pickle
import cv2
from matplotlib import pyplot as plt

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", required=True,
	help="path to trained serialized model")
ap.add_argument("-l", "--label-bin", required=True,
	help="path to  label binarizer")
ap.add_argument("-i", "--input", required=True,
	help="path to our input video")
ap.add_argument("-o", "--output", required=True,
	help="path to our output video")
ap.add_argument("-s", "--size", type=int, default=128,
	help="size of queue for averaging")
args = vars(ap.parse_args())

# load the trained model and label binarizer from disk
# 从命令行的声明载入训练好的模型以及二值化
print("[INFO] loading model and label binarizer...")
model = load_model(args["model"])
lb = pickle.loads(open(args["label_bin"], "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
# 初始化Q为一个双向序列，deque的尺寸由size决定
Q = deque(maxlen=args["size"])

# initialize the video stream, pointer to output video file, and
# frame dimensions
# 使用opencv的VideoCapture类读取视频流，并初始化视频写入类
vs = cv2.VideoCapture(args["input"])
writer = None
(W, H) = (None, None)

# loop over frames from the video file stream
# 循环抓取被测试视频的帧
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	# 如果没有抓取到视频帧，则退出
	if not grabbed:
		break

	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]

	# clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction 归一化操作
	output = frame.copy()
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	frame = cv2.resize(frame, (224, 224)).astype("float32")
	frame -= mean

	# make predictions on the frame and then update the predictions
	# queue
	# 新建一维用来储存每一帧对应的分类结果
	preds = model.predict(np.expand_dims(frame, axis=0))[0]
	# 将预测值加载Q后面
	Q.append(preds)

	# perform prediction averaging over the current history of
	# previous predictions
	# 先平均到K个概率输出值，在取最大
	results = np.array(Q).mean(axis=0)
	i = np.argmax(results)
	label = lb.classes_[i]

	# draw the activity on the output frame
	text = "activity: {}".format(label)
	cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
		1.25, (0, 255, 0), 5)

	# check if the video writer is None
	if writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 30,
			(W, H), True)

	# write the output frame to disk
	writer.write(output)

	# show the output image
	# 避免cannot connect to X server，此处使用matplotlib
	# cv2.imshow("Output", output)
  plt.imshow(output)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

In [0]:
!sudo apt-get install tree

In [0]:
!tree --dirsfirst --filelimit 50

.
├── data
│   ├── badminton [938 entries exceeds filelimit, not opening dir]
│   ├── baseball [746 entries exceeds filelimit, not opening dir]
│   ├── basketball [495 entries exceeds filelimit, not opening dir]
│   ├── boxing [705 entries exceeds filelimit, not opening dir]
│   ├── chess [481 entries exceeds filelimit, not opening dir]
│   ├── cricket [715 entries exceeds filelimit, not opening dir]
│   ├── fencing [635 entries exceeds filelimit, not opening dir]
│   ├── football [799 entries exceeds filelimit, not opening dir]
│   ├── formula1 [687 entries exceeds filelimit, not opening dir]
│   ├── gymnastics [719 entries exceeds filelimit, not opening dir]
│   ├── hockey [572 entries exceeds filelimit, not opening dir]
│   ├── ice_hockey [715 entries exceeds filelimit, not opening dir]
│   ├── kabaddi [454 entries exceeds filelimit, not opening dir]
│   ├── models
│   │   ├── res50-stage-1.pth
│   │   ├── res50-stage-2.pth
│   │   ├── res50-stage-3.pth
│   │   ├── stage-1.pth
│   │

In [0]:
!python predict_video.py --model output/activity.model \
	--label-bin output/lb.pickle \
	--input example_clips/lifting.mp4 \
	--output output/lifting_128frame.avi \
	--size 128  

In [0]:
!python predict_video.py --model output/activity.model \
	--label-bin output/lb.pickle \
	--input example_clips/tennis.mp4 \
	--output output/tennis_128frames_smoothened.avi \
	--size 128